In [386]:
import pandas as pd
import os
import numpy as np
pd.set_option('display.max_columns', 500)

In [420]:
counter=34
tire_csvs = np.intersect1d(np.array(range(34)),[int(x[:-4]) for x in os.listdir('stints_frames')])
source_conflict = [0,14,20,25]

In [421]:
np.setdiff1d(tire_csvs,source_conflict)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 15, 16, 17, 18,
       19, 21, 22, 23, 24, 26, 28, 29, 30, 31, 32, 33])

In [419]:
len(tire_csvs)

33

In [402]:
race_df=pd.read_csv('clean_frames/race_df_25.csv')

In [403]:
driver_df = pd.read_csv('drivers_numbers.csv')
d5={}
for race_id in tire_csvs:
    tires_df = pd.read_csv('stints_frames/'+str(race_id)+'.csv',header=None)    
    for i in range(1,tires_df.shape[1]):
        tires_df.rename(columns={i:'stint_'+str(i)},inplace=True)
        if i>0:
            tires_df['stint_'+str(i)][tires_df['stint_'+str(i)].notna()] = tires_df['stint_'+str(i)][tires_df['stint_'+str(i)].notna()].apply(np.vectorize(lambda x: x.split(' (')[0]))
    tires_df.rename(columns={0:'Driver'},inplace=True)
    tires_df = tires_df.merge(driver_df, how='left', on='Driver')
    d5['tires_df_'+str(race_id)] = tires_df
    tires_df.to_csv('clean_frames/tires_df_'+str(race_id)+'.csv')

In [404]:
tires_df = d5['tires_df_25']

In [408]:
new_df = new_df.merge(tires_df,how='outer',on='NO')

new_df['tyres']=np.nan

for stint in new_df['stint'].unique():
    new_df['tyres'][new_df['stint']==stint] = new_df['stint_'+str(int(stint))][new_df['stint']==stint]

/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


KeyError: 'stint_4'

In [406]:
# driver_df = pd.read_csv('drivers_numbers.csv')

# tires_df = d5['tires_df_'+str(0)]

# for i in range(tires_df.shape[1]):
#     tires_df.rename(columns={i:'stint_'+str(i)},inplace=True)
# tires_df.rename(columns={'stint_0':'Driver'},inplace=True)

# tires_df = tires_df.merge(driver_df, how='left', on='Driver')

In [407]:
new_df=pd.read_csv('new_df.csv')

new_df.set_index(['NO','LAP'], inplace=True)

def reset_lap_counter(lap,pits):
#     print('lap',lap)
    return lap - np.append(np.intersect1d(pits,np.arange(1,lap)),0).max()

new_df['d_stop'] = new_df['STOP']>0
new_df['target'] = np.nan

new_df['stint'] = new_df['STOP']
new_df['laps_since_pit'] = new_df.index.get_level_values('LAP')
for _num in new_df.index.get_level_values('NO').unique():
    pits = new_df.index.get_level_values('LAP')[(new_df['d_stop'])&(new_df.index.get_level_values('NO')==_num)].values
    new_df['stop_count'][new_df.index.get_level_values('NO')==_num] = new_df['stop_count'][new_df.index.get_level_values('NO')==_num].ffill()
    new_df['target'][(new_df.index.get_level_values('NO') == _num)] = new_df['d_stop'][(new_df.index.get_level_values('NO') == _num)].shift(-1)
#     new_df['stint'][new_df.index.get_level_values('NO')==_num] = new_df['stint'][new_df.index.get_level_values('NO')==_num].bfill()#.ffill()
#     new_df['stint'][new_df.index.get_level_values('NO')==_num] = new_df['stint'][new_df.index.get_level_values('NO')==_num].fillna(new_df['stint'][new_df.index.get_level_values('NO')==_num].max()+1)
    new_df['laps_since_pit'][new_df.index.get_level_values('NO')==_num] = new_df['laps_since_pit'][new_df.index.get_level_values('NO')==_num].apply(lambda x: reset_lap_counter(x,pits))
new_df['stint']=new_df['stop_count'].astype('int32')+1

# new_df['stop_count'] = new_df['STOP']
# new_df['stop_count'][(new_df.index.get_level_values(1)==1) & (new_df['STOP']!=1)] = 0
# new_df['stop_count'][(new_df.LAP==1) & (new_df['STOP']!=1)] = 0

/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/guido/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A

In [232]:
# def tires(stint,NO):#,LAP):
#     print(tires_df['stint_'+str(stint)][tires_df['NO']==NO])

In [268]:
# def tires(x,_num):
#     return tires_df['stint_'+str(x)][tires_df['NO']==_num]

In [269]:
# for _num in new_df.index.get_level_values('NO').unique():
#     new_df['tyres'] = new_df['stint'].apply(lambda x: tires(x,_num))

In [342]:
test_df.columns

Index(['NO', 'GAP', 'TIME', 'race_id', 'cumtime', 'progress', 'temp_lap', '5',
       '7', '6', '33', '19', '44', '55', '27', '3', '14', '11', '22', '30',
       '94', '9', '12', '77', '8', '88', '21', '20', 'DRIVER', 'ENTRANT',
       'TIME OF DAY', 'STOP', 'DURATION', 'TOTAL TIME', 'd_stop', 'target',
       'laps_since_pit', 'stop_count', 'stint', 'tyres', 'Driver', 'stint_1',
       'stint_2', 'stint_3', 'stint_4'],
      dtype='object')

In [306]:
# test_df[test_df['NO']==94]

In [307]:
# new_df[new_df.index.get_level_values(0)==94]

In [254]:
new_df['tyres'][new_df.index.get_level_values('NO')==22].shape #= 
# (tires_df['stint_'+new_df['stint'].astype('str')][tires_df['NO']==22]).T

(56,)

In [194]:
tires(2,44)

1    Medium (41)
Name: stint_2, dtype: object


In [162]:
new_df['tyres'] = new_df['stint'].apply(tires)

In [336]:
new_df[new_df['stint']==4].index.get_level_values(0).unique()
# new_df[(new_df['stint']==3)&(new_df.index.get_level_values(0)==22)]

Int64Index([20, 9, 22, 55, 33, 94, 5, 3], dtype='int64', name='NO')

In [115]:
def tires_num(x):
    if type(x)==str:
        if 'Super soft' in x:
            return 0
        elif 'Soft' in x:
            return 1
        elif 'Medium'in x:
            return 2
#     else:
#         return None

In [409]:
tires_df

,Driver,stint_1,stint_2,stint_3,NO
0,Sebastian Vettel,Super soft,Soft,NaN,5
1,Valtteri Bottas,Super soft,Soft,NaN,77
2,Kimi Raikkonen,Super soft,Soft,NaN,7
3,Lewis Hamilton,Soft,Super soft,NaN,44
4,Max Verstappen,Super soft,Soft,Super soft,33
5,Daniel Ricciardo,Soft,Soft,Super soft,3
6,Felipe Massa,Super soft,Soft,NaN,19
7,Fernando Alonso,Super soft,Soft,NaN,14
8,Sergio Perez,Super soft,Soft,NaN,11
9,Nico Hulkenberg,Super soft,Soft,NaN,27


In [398]:
tires_df

,Driver,stint_1,stint_2,stint_3,stint_4,stint_5,NO
0,Lewis Hamilton,Intermediate,Ultra soft,NaN,NaN,NaN,44
1,Daniel Ricciardo,Intermediate,Intermediate,Ultra soft,NaN,NaN,3
2,Valtteri Bottas,Intermediate,Ultra soft,NaN,NaN,NaN,77
3,Carlos Sainz Jnr,Intermediate,Super soft,NaN,NaN,NaN,55
4,Sergio Perez,Wet,Intermediate,Ultra soft,NaN,NaN,11
5,Jolyon Palmer,Wet,Intermediate,Ultra soft,NaN,NaN,30
6,Stoffel Vandoorne,Wet,Intermediate,Ultra soft,NaN,NaN,2
7,Stoffel Vandoorne,Wet,Intermediate,Ultra soft,NaN,NaN,47
8,Lance Stroll,Intermediate,Ultra soft,NaN,NaN,NaN,18
9,Romain Grosjean,Intermediate,Ultra soft,NaN,NaN,NaN,8


In [375]:
d5.keys()

dict_keys(['tires_df_0', 'tires_df_1', 'tires_df_2', 'tires_df_3', 'tires_df_4', 'tires_df_5', 'tires_df_6', 'tires_df_7', 'tires_df_8', 'tires_df_9', 'tires_df_10', 'tires_df_11', 'tires_df_12', 'tires_df_13', 'tires_df_14', 'tires_df_15', 'tires_df_16', 'tires_df_17', 'tires_df_18', 'tires_df_19', 'tires_df_20', 'tires_df_21', 'tires_df_22', 'tires_df_23', 'tires_df_24', 'tires_df_25', 'tires_df_26', 'tires_df_28', 'tires_df_29', 'tires_df_30', 'tires_df_31', 'tires_df_32', 'tires_df_33'])

In [399]:
ref_df = pd.read_csv('which_race.csv', header=0)
ref_df[ref_df['race_id']==20]

,race_id,year,race_in_year,track
20,20,2017,13,Singapore


In [376]:
d5['tires_df_14']

,Driver,stint_1,stint_2,stint_3,stint_4,NO
0,Lewis Hamilton,Ultra soft,Soft,Soft,NaN,44
1,Max Verstappen,Super soft,Soft,NaN,NaN,33
2,Kimi Raikkonen,Super soft,Soft,NaN,NaN,7
3,Nico Rosberg,Ultra soft,Soft,Super soft,NaN,6
4,Daniel Ricciardo,Super soft,Soft,Ultra soft,NaN,3
5,Jenson Button,Ultra soft,Soft,Soft,NaN,22
6,Romain Grosjean,Super soft,Soft,NaN,NaN,8
7,Carlos Sainz Jnr,Super soft,Soft,Soft,NaN,55
8,Valtteri Bottas,Ultra soft,Soft,Super soft,NaN,77
9,Pascal Wehrlein,Ultra soft,Ultra soft,Soft,NaN,94


In [410]:
ref_df = pd.read_csv('which_race.csv', header=0);
ref_df[ref_df['race_id']==25]

,race_id,year,race_in_year,track
25,25,2017,18,São Paulo
